In [1]:
import pandas as pd
import numpy as np


pd.options.display.max_columns = 999
pd.options.display.max_rows = 999
pd.options.display.max_colwidth = 100
import warnings
warnings.filterwarnings('ignore')

from tabula import read_pdf
import camelot

import os

## Source
 - [Boletim](http://www.saude.sp.gov.br/ses/perfil/cidadao/homepage/outros-destaques/covid-19-plano-de-contingencia-boletins-diarios-e-outras-informacoes.)


In [27]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
import time
from os import listdir


In [50]:
download_button

[]

## Source
 - [Boletim](http://www.saude.sp.gov.br/ses/perfil/cidadao/homepage/outros-destaques/covid-19-plano-de-contingencia-boletins-diarios-e-outras-informacoes.)


In [87]:
file_name = "Relação-de-Compras-COVID19_completa_2804"

dd = camelot.read_pdf(f'../data/orcamento/pdf/{file_name}.pdf', pages='all')

In [88]:
df = pd.DataFrame()

for i in range(len(dd)):
    df = pd.concat([df,dd[i].df], axis=0)
    
df = df.reset_index(drop=True)

In [89]:
cols = ['orgao','descricao','valor','n_processo','cnpj','fornecedor','vigencia']
df.columns = cols
df = df[df['descricao']!='']
df['valor'] = df['valor'].apply(lambda x: x.replace('R$','').replace('\n','').strip().replace('.','').replace(',','.')).astype(float)
df['orgao'] = df['orgao'].apply(lambda x: x.replace('\n','').strip())



df['last_update'] = '2020-04-28'

In [92]:
print(df['valor'].sum())
print(len(df),"\n")

912767442.4499998
140 



In [93]:
df.to_csv(f'../data/orcamento/csv/{file_name}.csv', index=False)
df.to_excel(f'../data/orcamento/csv/{file_name}.xlsx', index=False)

# Data processing

In [141]:
#Merge all files
path = '../data/orcamento/csv/'
files = os.listdir(path)

df_final = pd.read_csv(f"{path}{files[0]}")
data = max(df_final['last_update'])
df_final[f'valor_{data}'] = df_final['Valor Total']
rename_cols = {'Descrição':f'Descrição_{data}',
               'Vigência (180 dias)':f'Vigência (180 dias)_{data}'} 
df_final = df_final.rename(columns=rename_cols)

for file in files[1:]:
    dff = pd.read_csv(f"{path}{file}")
    print(file)
    print(dff['Valor Total'].sum())
    print(len(dff),"\n")
    
    data = max(dff['last_update'])
    
    dff[f'valor_{data}'] = dff['Valor Total']
    rename_cols = {'Descrição':f'Descrição_{data}',
               'Vigência (180 dias)':f'Vigência (180 dias)_{data}'} 
    dff = dff.rename(columns=rename_cols)

    
    


rel_compras-covid19_publicacao_3v.csv
309661423.25
107 



In [142]:
df_final = pd.merge(df_final, dff, on=['Valor Total','Nº Processo (SES-PRC)','CNPJ'], how='outer')


In [143]:
df_final['Valor Total'].sum()

332681090.38

In [2]:
# df_final

In [98]:
cols = ['Descrição','Valor Total', 'Nº Processo (SES-PRC)', 'CNPJ']


df_final = df_final.drop_duplicates(subset=cols, keep='first')

In [99]:
print(df_final['Valor Total'].sum())
print(len(df_final),"\n")

485974413.63
208 



In [107]:
df_final.sort_values(by='Valor Total').to_excel('lici.xlsx', index=False)

## CNPJS

In [191]:
file_name = "Relação-de-Compras-COVID19_completa_2804"

df = pd.read_csv('../data/orcamento/csv/Relação-de-Compras-COVID19_completa_2804.csv')
df['fornecedor'] = df['fornecedor'].apply(lambda x: x[:-1] if x[-1]=='.' else x).str.replace('VEDANA COM. DE PROD. E SERVIÇOS PARA SAÚDE EIRELLI-EPP','VEDANA COM. DE PRODUTOS E SERV. P/ SAUDE LTDA')

df['id'] = df['cnpj'].str.replace('.','').str.replace('-','').str.replace('/','')
df['descricao'] = df['descricao'] + '. Valor: ' +  df['valor'].astype(str)
df_values = df.groupby(by=['id','cnpj','fornecedor'], as_index=False).agg({'descricao':'\n\n'.join, 'valor':sum})
df_values = df_values.rename(columns={'cnpj':'CNPJ','valor':'valor total'})

In [192]:
url = f'http://compras.dados.gov.br/fornecedores/doc/fornecedor_pj/'

df_final = pd.DataFrame()
for cnpj in df['id'].unique():
    url_csv = url + f'{cnpj}.csv'

    
    try:
        dd = pd.read_csv(url_csv)
        print(url_csv)
        dd = dd.drop(columns= ['Id','CNPJ','Unidade Cadastradora','Ramo do Negócio','Porte da Empresa','CNAE Secundário','Caixa Postal','Recadastrado'])
        dd['id'] = cnpj
        
    except:
        dd = pd.DataFrame([cnpj],columns=['id'])
        
        print(cnpj, 'nao existe')

    
    df_final = pd.concat([df_final,dd],axis=0)

df_final = df_final.sort_values(by='Razão Social')


http://compras.dados.gov.br/fornecedores/doc/fornecedor_pj/11034934000160.csv
http://compras.dados.gov.br/fornecedores/doc/fornecedor_pj/00647935000164.csv
http://compras.dados.gov.br/fornecedores/doc/fornecedor_pj/04192876000138.csv
http://compras.dados.gov.br/fornecedores/doc/fornecedor_pj/02456955000183.csv
http://compras.dados.gov.br/fornecedores/doc/fornecedor_pj/04274988000138.csv
http://compras.dados.gov.br/fornecedores/doc/fornecedor_pj/04027894000750.csv
http://compras.dados.gov.br/fornecedores/doc/fornecedor_pj/03951140000133.csv
http://compras.dados.gov.br/fornecedores/doc/fornecedor_pj/03436772000169.csv
http://compras.dados.gov.br/fornecedores/doc/fornecedor_pj/94516671000153.csv
http://compras.dados.gov.br/fornecedores/doc/fornecedor_pj/07118264000193.csv
http://compras.dados.gov.br/fornecedores/doc/fornecedor_pj/71957310000147.csv
http://compras.dados.gov.br/fornecedores/doc/fornecedor_pj/48791685000168.csv
http://compras.dados.gov.br/fornecedores/doc/fornecedor_pj/52202

In [196]:
ddd = pd.merge(df_values, df_final, on='id', how='outer').sort_values(by='Razão Social').drop(columns=['id'])

In [199]:
ddd.to_excel('../data/orcamento/csv/lista_cnpjs.xlsx', index=False)

## Cidades

In [4]:
import pandas as pd
import numpy as np


pd.options.display.max_columns = 999
pd.options.display.max_rows = 999
pd.options.display.max_colwidth = 100
import warnings
warnings.filterwarnings('ignore')

from tabula import read_pdf
import camelot

import os

## Fonte
 - [Boletim](https://www.sjc.sp.gov.br/servicos/saude/coronavirus/atos-e-despesas/)

In [5]:
#sjc

file_name = "autorizacao-de-fornecimento-covid-ate-24042020"

dd = camelot.read_pdf(f'../data/orcamento/pdf/{file_name}.pdf', pages='all')

In [6]:
df = pd.DataFrame()

for i in range(len(dd)):
    df = pd.concat([df,dd[i].df], axis=0)
    
df = df.reset_index(drop=True)

In [10]:
cols = ['material','descricao','quantidade_af','unidade','valor_unitario','valor_total','numero_af','data','modalidade_de_compra','razao_social','unidade_orcamentaria','codigo_dotacao_orcamentaria']
df.columns = cols
df = df[df['material']!='MATERIAL']

df['valor_unitario'] = df['valor_unitario'].apply(lambda x: x.replace('R$','').replace('\n','').strip().replace('.','').replace(',','.'))
df['valor_total'] = df['valor_total'].apply(lambda x: x.replace('R$','').replace('\n','').strip().replace('.','').replace(',','.'))

df = df.replace('\n','')

In [13]:
df.to_csv(f'../data/orcamento/csv/sjc/{file_name}.csv', index=False)

## Fonte
 - [Boletim](http://www.osasco.sp.gov.br/portal-da-transparencia)

In [ ]:
#osasco

file_name = "autorizacao-de-fornecimento-covid-ate-24042020"

dd = camelot.read_pdf(f'../data/orcamento/pdf/{file_name}.pdf', pages='all')